In [1]:
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

seed=3
np.random.seed(seed)
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

Using TensorFlow backend.


In [2]:
labels_df = pd.read_csv('product_attributes_updated.csv', encoding='utf-8', 
                    dialect='excel', lineterminator='\n')

In [ ]:
for ColumnName in ['Type']:
    labels_df[ColumnName].value_counts().plot(kind="bar")
    plt.show()

In [3]:
 def image2np_array( infilename ) :
    img = Image.open( infilename )
    img.load()
    data = np.asarray( img, dtype="int32" )
    return data

x_train_list=[]
y_train_list=[]

one_hot=pd.get_dummies(labels_df['Type'])

for row in labels_df.itertuples():
    if pd.notnull(row[7]):
        try:
            img = image2np_array('..\\ml_150x150_m\\'+row[1]+'.jpg').reshape(150,150,1)
        except IOError:
            continue
        x_train_list.append(img)
        y_train_list.append(one_hot.loc[row[0],:].values)

In [4]:
x_train=np.array(x_train_list)
y_train=np.array(y_train_list)
len(y_train)==len(x_train)
n_of_training_samples=int(len(x_train)*0.8)
x_validation=x_train[n_of_training_samples:]
y_validation=y_train[n_of_training_samples:]
x_train=x_train[:n_of_training_samples]
y_train=y_train[:n_of_training_samples]

In [5]:
epochs = 10
batch_size = 32

input_shape = (150, 150, 1)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5,seed=seed))
model.add(Dense(80))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
metrics=['accuracy'])

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    zoom_range=0.2,
    horizontal_flip=True)

model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size),
                    validation_data=(x_validation,y_validation),
                    steps_per_epoch=len(x_train) / batch_size, epochs=epochs)

Epoch 1/10
857/856 [==============================] - 50s - loss: 2.4297 - acc: 0.3641 - val_loss: 9.3202 - val_acc: 0.4061
Epoch 2/10
857/856 [==============================] - 47s - loss: 1.8370 - acc: 0.4985 - val_loss: 8.7097 - val_acc: 0.4477
Epoch 3/10
857/856 [==============================] - 47s - loss: 1.6230 - acc: 0.5520 - val_loss: 11.2667 - val_acc: 0.2868
Epoch 4/10
857/856 [==============================] - 47s - loss: 1.4642 - acc: 0.5910 - val_loss: 13.4309 - val_acc: 0.1596
Epoch 5/10
857/856 [==============================] - 47s - loss: 1.3575 - acc: 0.6145 - val_loss: 11.0271 - val_acc: 0.3012
Epoch 6/10
857/856 [==============================] - 47s - loss: 1.2900 - acc: 0.6293 - val_loss: 13.8876 - val_acc: 0.1323
Epoch 7/10
857/856 [==============================] - 47s - loss: 1.2296 - acc: 0.6420 - val_loss: 13.1286 - val_acc: 0.1813
Epoch 8/10
857/856 [==============================] - 46s - loss: 1.1738 - acc: 0.6614 - val_loss: 11.1731 - val_acc: 0.2907
Ep

In [7]:
model.save('type_classifier.h5')